## Reading data using pandas

In [3]:
import pandas as pd
compasData = pd.read_csv('Datasets/compas-scores-two-years.csv')

In [4]:
compasData


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


## Data Cleaning and Preprocessing

1- Keeping only defendants who were assigned risk compas scores within 30 days of their arrests 

In [5]:
compasData = compasData[(compasData['days_b_screening_arrest'] <= 30) | (compasData['days_b_screening_arrest'] >= -30)]                     
print(len(compasData))                        

6907


2- Binarizing the protected attribute (race)

In [6]:
# selecting 2 groups only: African-American (black defendants) and Caucasian (white defendants)
compasData = compasData[(compasData['race'] == "African-American") | (compasData['race'] == "Caucasian")]                     
print(len(compasData))    

5915


In [7]:
# renaming both groups
compasData.replace('African-American', 'Black defendants', inplace=True)
compasData.replace('Caucasian', 'White defendants', inplace=True)

In [9]:
# variables to be used  
compasData = compasData [['sex','race','age_cat','priors_count','v_decile_score']]

In [11]:
from sklearn.preprocessing import LabelEncoder

income_encoder = LabelEncoder()
features = ['sex', 'race','age_cat'] # for sex (Female:0, Male:1), for race (Black defendants:0, Whites defendants:1), #25 - 45 (0), Less than 25 (2), Greater than 45 (1)   

for att in features:
    compasData[att] = income_encoder.fit_transform(compasData[att])
# binarizing the priors_counts    
compasData['priors_count'] = compasData.apply(lambda row: 0 if row['priors_count'] <= 2 else 1 , axis = 1)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [65]:
compasData

,sex,race,age_cat,priors_count,v_decile_score
1,1,0,0,0,1
2,1,0,2,1,3
6,1,1,0,1,2
8,0,1,0,0,1
9,1,1,2,0,5
...,...,...,...,...,...
7207,1,0,0,0,2
7208,1,0,2,0,9
7209,1,0,2,0,5
7210,1,0,2,0,5


## Binarizing the target and generating the different datasets¶

In [13]:
compasCopy = compasData.copy()
for th in [1,3,5]:
    compasData['v_decile_score'] = compasData['v_decile_score'].apply(lambda x: 1 if x > th else 0) #float(th) , axis = 1) #label_cutoffs(x,i), axis=1)    
    compasData.to_csv('Datasets/compas_'+ str(th) +'.csv', index=False) 
    compasData ['v_decile_score'] = compasCopy['v_decile_score']    

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
